In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-11-24 14:42:02.578579: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 14:42:03.773462: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-24 14:42:04.103071: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 14:42:04.664285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 14:42:04.918192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 14:42:06.200195: I tensorflow/core/platform/cpu_feature_gu

In [3]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [4]:
data_loader = DataLoader(1, True, 23)
train_df, test_df = data_loader.create_train_test_df(True, True, True, True, 1)
train_df

All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64


,Text,Numerical Rating,score,index,Category,Synthetic,predict
9157,"1924/1925: PDO changed to a ""warm"" phase.",3,NaN,NaN,NaN,NaN,NaN
5796,"""Hurricane Harvey was year's costliest U.S. di...",3,NaN,NaN,NaN,NaN,NaN
9453,Oil refineries also have negative environmenta...,3,NaN,NaN,NaN,NaN,NaN
6853,"Robert Watson found this ""very disappointing"" ...",3,NaN,NaN,NaN,NaN,NaN
6593,"""Climate change could impact the poor much mor...",3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2507,One analysis found that a 2.% share in fossil ...,1,0.886674,4900.0,1_1,False,NaN
7697,Models also suggest that the ozone depletion/e...,3,NaN,NaN,NaN,NaN,NaN
4022,"Today, April 22, as the Biden-Harris Administr...",3,NaN,NaN,-1,False,False
3674,Distributed solar enthusiasts (who favor photo...,1,0.891835,712.0,1_1,False,NaN


In [5]:
test_df

,Text,Numerical Rating,score,index,Category,Synthetic,predict
0,A recent MIT study concludes that under a poli...,1,0.887588,2405.0,1_1,False,NaN
1,As reported in The Heartland Institute's Envir...,1,0.885880,1949.0,1_1,False,NaN
2,According to the French financial daily Les Ec...,1,0.867037,2005.0,1_1,False,NaN
3,Policymakers have also offered a slew of subsi...,1,0.808251,2577.0,1_1,False,NaN
4,This graph below from Bishop Hill shows that i...,1,0.847182,3399.0,1_1,False,NaN
...,...,...,...,...,...,...,...
1722,"Between 1990 and 2023, annual emissions of SO2...",1,NaN,NaN,NaN,False,NaN
1723,"Compared to 2022, the 2023 data show a 25% dec...",1,NaN,NaN,NaN,False,NaN
1724,Reported power plant emissions increased drama...,1,NaN,NaN,NaN,False,NaN
1725,There is an decrease in emmisions as nations s...,1,NaN,NaN,NaN,False,NaN


In [ ]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
i = 0.6
for param in params:
    print("Running experiment ", i)
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']

    train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    if use_only_card:
        print("using card")
        train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)

#     train_df, test_df = data_loader.create_half_dataset()
    # Sample i% of train_df and i% of test_df
    train_df = train_df.sample(frac=i, random_state=int(param['random_seed']))
    test_df = test_df.sample(frac=i, random_state=int(param['random_seed']))

    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_df.to_csv(f'./Iterations/cluster_df_{i}.csv')
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': size_of_dataset,
        'use_only_CARD': use_only_card
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)
    results_df = pd.json_normalize(results)
    results_df.to_csv(f'./Iterations/results_df_{i}.csv')
    results = []
    i += 0.2

All data sources are being used
Number of experiments to run:  3
Running experiment  0.6
Numerical Rating
3    3256
1      28
Name: count, dtype: int64
Numerical Rating
3    771
1     51
Name: count, dtype: int64
using card
All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  1036
Getting embeddings for batch  50  out of  1036
Getting embeddings for batch  100  out of  1036
Getting embeddings for batch  150  out of  1036
Getting embeddings for batch  200  out of  1036
Getting embeddings for batch  250  out of  1036
Getting embeddings for batch  300  out of  1036
Getting embeddings for batch  350  out of  1036
Getting embeddings for batch  400  out of  1036
Getting embeddings for batch  450  out of  1036
Getting embeddings for batch  500  out of  1036
Getting embeddings for batch  550  out of  1036
Getting embedding

/home/codespace/.local/lib/python3.10/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/home/codespace/.local/lib/python3.10/site-packages/pynndescent/pynndescent_.py:962: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


Running hdbscan...
breaking further...
98
107
107
107
111
111
111
111
113
113
113
113
113
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
117
117
117
120
120
120
122
122
122
122
122
122
143
143
143
143
143
143
145
145
145
145
147
149
149
149
149
149
149
149
149
149
cluster  predict  veracity
-1       False     3          1948
                   1          1086
         True     -1           606
 1       False     3             5
 9       False     3             5
                              ... 
 151     False     1             2
 152     False     3            13
                   1             7
 153     False     1             6
                   3             3
Name: count, Length: 132, dtype: int64
breaking further...
153
155
155
155
155
155
155
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
157
159
159
159
159
159
159
159
159
159
clu

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


predict
True    19
Name: count, dtype: int64
                                                   text  veracity  predict  \
979   The economic cost of climate policy for the ma...         1     True   
980   “Believing CO2 controls the climate is pretty ...         1     True   
986   The highest temperature ever recorded in Oxfor...         3     True   
990   When the Czechoslovak government in London nee...         1     True   
996   By 1929, the Great Depression arrived, causing...         3     True   
997   In 2008, Britain's Prince Charles said we only...         1     True   
1002  In 2002, the House passed trade promotion auth...         1     True   
1003  Obama's approach of selectively combining regu...         3     True   
1006  h/t Neil.. LaFramboise further found that one ...         1     True   
1007  The Koch brothers bought NPR and stopped the b...         1     True   
1009  In areas with high soot production, such as ru...         3     True   
1011  "U.S. Report 

...

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ERROR DEBUG
veracity
3    530
Name: count, dtype: int64
predicted_veracity
1    530
Name: count, dtype: int64
sad...
ERROR DEBUG
veracity
1    506
Name: count, dtype: int64
predicted_veracity
3    498
1      8
Name: count, dtype: int64
ERROR DEBUG
veracity
3    11
Name: count, dtype: int64
predicted_veracity
1    11
Name: count, dtype: int64
sad...
ERROR DEBUG
veracity
1    8
Name: count, dtype: int64
predicted_veracity
1    8
Name: count, dtype: int64
Running experiment  0.8
Numerical Rating
3    3256
1      28
Name: count, dtype: int64
Numerical Rating
3    771
1     51
Name: count, dtype: int64
using card
All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  1382
Getting embeddings for batch  50  out of  1382
Getting embeddings for batch  100  out of  1382
Getting embeddings for batch  150  out of  1382
Getting embeddings for batch  200  out of  1382
Getting embeddings for batch  250  out of  1382
Getting embeddings for batch  300  out of  1382
Getting embeddings for batch  350  out of  1382
Getting embeddings for batch  400  out of  1382
Getting embeddings for batch  450  out of  1382
Getting embeddings for batch  500  out of  1382
Getting embeddings for batch  550  out of  1382
Getting embeddings for batch  600  out of  1382
Getting embeddings for batch  650  out of  1382
Getting embeddings for batch  700  out of  1382
Getting embeddings for batch  750  out of  1382
Getting embeddings for batch  800  out of  1382
Getting embeddings for batch  850  out of  1382
Getting embeddings for batch  900  out of  1382
Getting embeddings for batch  950  out of  1382
Getting embed

/home/codespace/.local/lib/python3.10/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/home/codespace/.local/lib/python3.10/site-packages/pynndescent/pynndescent_.py:962: UserWarning: Failed to correctly find n_neighbors for some samples. Results may be less than ideal. Try re-running with different parameters.
  warn(


In [4]:
results_df = pd.json_normalize(results)
# Get current date and time
from datetime import datetime
now = datetime.now()
results_df
results_df.to_csv(f'results_${str(now)}.csv', index=False)

In [11]:
score = best_estimator.score([], [])


predict
True    2854
Name: count, dtype: int64
                                                   text  veracity  predict  \
0     Among the political idiosyncrasies and trendse...        -1     True   
1     Mr. Sanders said that none of the Republican p...        -1     True   
2     Climatology deals with the aggregate data that...        -1     True   
3     "Shifts in ENSO coupling processes under globa...        -1     True   
4     In this context, John Tierney of The New York ...        -1     True   
...                                                 ...       ...      ...   
2849  Terrestrial television is a type of television...        -1     True   
2850  Fifty-seven predictions were made regarding 19...        -1     True   
2851  Saving Money, Reducing Emissions, , EPA estima...        -1     True   
2852  It may be the largest forest insect blight eve...        -1     True   
2853  to Kyoto only represent 15 per cent of global ...        -1     True   

      predicted_

ValueError: No claims in cluster_df

In [11]:
# Save cluster_df from results[0] to csv
from datetime import datetime

now = datetime.now()
results[0]['metrics']['cluster_df'].to_csv(f'cluster_df_${str(now)}.csv', index=False)

In [7]:
best_estimator.clusters_df
# Filter clusters_df to where cluster is not -1 and predicted_cluster is 4
best_estimator.clusters_df[(best_estimator.clusters_df['cluster'] != -1) & (best_estimator.clusters_df['predicted_veracity'] == 4)]
# Filter to cluster 144
best_estimator.clusters_df[best_estimator.clusters_df['cluster'] == 144]

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy
74,"Compared to 2022, the 2023 data show a 25% dec...",1,True,4,"[2.463346242904663, 2.7957773208618164, 9.9586...",144,0,5,0.0
378,Improvements in existing plants reduced steel-...,3,True,4,"[2.42880916595459, 2.778208017349243, 9.946139...",144,0,5,0.0
438,Reported power plant emissions increased drama...,1,True,4,"[2.5143368244171143, 2.7544777393341064, 9.951...",144,0,5,0.0
649,"In 2022, reported emissions from large industr...",1,True,4,"[2.4696571826934814, 2.7592506408691406, 9.971...",144,0,5,0.0
862,"Between 1990 and 2023, annual emissions of SO2...",1,True,4,"[2.4708685874938965, 2.7929885387420654, 9.978...",144,0,5,0.0


In [ ]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
df_with_two_dimens